In [14]:
%load_ext autoreload
%autoreload 2
from datasets import DatasetDict
from pysentimiento.lince.ner import load_datasets


train_dataset, dev_dataset, test_dataset = load_datasets("es")

lince_ner = DatasetDict(
    train=train_dataset,
    validation=dev_dataset,
    test=test_dataset,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reusing dataset lince (/home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-96b92c9276007a0d.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-5e10b10d764d61ca.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/ner_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-168c9ba86661591e.arrow


  0%|          | 0/33611 [00:00<?, ?ex/s]

  0%|          | 0/10085 [00:00<?, ?ex/s]

  0%|          | 0/23527 [00:00<?, ?ex/s]

In [15]:
train_dataset[101]["words"]

['@usuario', 'JA', '!', 'yo', 'solo', 'hise', 'la', 'mono']

In [16]:
idx = 11065
print(lince_ner["train"][idx]["ner"])
print(lince_ner["train"][idx]["labels"])
lince_ner["train"][idx].keys(), lince_ner["train"][idx]

['O', 'O', 'O', 'B-OTHER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O']
[0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 11, 0, 0, 0, 0, 0]


(dict_keys(['ner', 'words', 'lid', 'labels', 'idx']),
 {'ner': ['O',
   'O',
   'O',
   'B-OTHER',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-PER',
   'I-PER',
   'O',
   'B-PER',
   'O',
   'O',
   'O',
   'O',
   'O'],
  'words': ['Im',
   'stuck',
   'in',
   'bachata',
   'for',
   'the',
   'rest',
   'of',
   'the',
   'week',
   '...',
   'prince',
   'royce',
   'and',
   'romeo',
   'are',
   'killing',
   'it',
   '!',
   ':)'],
  'lid': ['lang1',
   'lang1',
   'lang1',
   'lang2',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'other',
   'ne',
   'ne',
   'lang1',
   'ne',
   'lang1',
   'lang1',
   'lang1',
   'other',
   'other'],
  'labels': [0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 11, 0, 0, 0, 0, 0],
  'idx': 11065})

In [17]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer


device = "cuda" if torch.cuda.is_available() else "cpu"
max_length = 128

model_name = "pysentimiento/robertuito-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [20]:

words = lince_ner["train"][idx+2]["words"]

tokens = tokenizer(words, is_split_into_words=True)

inv_vocab = {v:k for k, v in tokenizer.get_vocab().items()}

print(words)
list(zip(tokens.word_ids(), [inv_vocab[t] for t in tokens["input_ids"]]))

['#mlrt', '>>>', 'has', 'me', 'dying', '😂👌']


[(None, '<s>'),
 (0, '▁#'),
 (0, 'm'),
 (0, 'l'),
 (0, 'rt'),
 (1, '▁'),
 (2, '▁has'),
 (3, '▁me'),
 (4, '▁dying'),
 (5, '▁'),
 (None, '</s>')]

In [21]:
from pysentimiento.lince.ner import label2id

def align_labels_with_tokens(labels, word_ids):
    """
    Tomado de https://huggingface.co/course/chapter7/2?fw=pt
    """
    labels = [label2id[l] for l in labels]
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels